In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import statsmodels.stats.proportion as sp
from scipy import stats
from scipy.stats import pearsonr

In [2]:
df = pd.read_csv('cleaned_lumoshive_data.csv')

/var/folders/8p/30y3pc9x2x5gq8qsq1lfw8480000gn/T/ipykernel_89229/2200081736.py:1: DtypeWarning: Columns (4,6,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_lumoshive_data.csv')


In [3]:
df.head()

,id,created_at,country,operator,service,source,msisdn,status,cycle,adnet,...,service_price,currency,profile_status,publisher,trxid,pixel,handset,browser,attempt_charging,success_billing
0,302164,2023-09-13 09:47:57,thailand,ais,459505104,sms,66988602911,not found,daily,not found,...,6.0,bath,active,not found,2.02309130707494e+19,not found,not found,not found,15,0
1,3521,2023-09-13 09:21:13,thailand,ais,459505104,sms,66822243080,not found,daily,not found,...,6.0,bath,active,not found,2.02309130707243e+19,not found,not found,not found,15,0
2,1,2023-09-13 09:20:20,thailand,ais,459505106,ivr,666131,not found,not found,not found,...,6.0,bath,active,not found,2.02308251402057e+19,not found,not found,not found,1,1
3,4999335,2023-09-13 14:00:22,thailand,ais,459505105,wap,66650020857,not found,daily,star,...,6.0,bath,active,star,2.02309130710521e+19,star,not found,not found,5,0
4,273196,2023-09-13 09:45:33,thailand,ais,459505104,sms,66635959286,not found,daily,not found,...,6.0,bath,active,not found,2.02309130701156e+19,not found,not found,not found,15,0


In [75]:
df.describe()

,id,revenue,service_price,attempt_charging,success_billing
count,3.656140e+05,365614.000000,365614.000000,365614.000000,365614.000000
mean,9.870090e+05,4351.895797,156.533388,21.520035,3.107988
std,4.171706e+06,15080.080037,770.647277,36.390434,12.170820
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,6.365700e+04,0.000000,6.000000,1.000000,0.000000
50%,1.926345e+05,0.000000,6.000000,15.000000,0.000000
75%,3.603850e+05,300.000000,6.000000,26.000000,1.000000
max,6.008518e+07,506000.000000,10000.000000,4015.000000,1363.000000


In [4]:
# Mapping dictionary for currency conversion
currency_mapping = {'bath': 441.23, 'cfa franc': 25.21040, 'lak': 0.75, 'kip': 0.75, 'rial': 4143.17, 'idr': 1, 'baisa': 4143.17, 'php': 279.48, 'baht':  449.06}

# Conversion function
def convert_to_rupiah(value, currency):
    conversion_rate = currency_mapping.get(currency, 1)
    return value * conversion_rate

# Apply the conversion to 'service_price' column and create a new column 'converted_service_price'
df['converted_service_price'] = df.apply(lambda row: convert_to_rupiah(row['service_price'], row['currency']), axis=1)

# Apply the conversion to 'revenue' column and create a new column 'converted_revenue'
df['converted_revenue'] = df.apply(lambda row: convert_to_rupiah(row['revenue'], row['currency']), axis=1)

# Display the updated DataFrame with new columns
print(df)
# Display the specific rows with the new columns
print(df[['service_price', 'currency', 'converted_service_price', 'revenue', 'converted_revenue']].head())

              id           created_at   country   operator    service source  \
0         302164  2023-09-13 09:47:57  thailand        ais  459505104    sms   
1           3521  2023-09-13 09:21:13  thailand        ais  459505104    sms   
2              1  2023-09-13 09:20:20  thailand        ais  459505106    ivr   
3        4999335  2023-09-13 14:00:22  thailand        ais  459505105    wap   
4         273196  2023-09-13 09:45:33  thailand        ais  459505104    sms   
...          ...                  ...       ...        ...        ...    ...   
365609     97584            not found  thailand        ais  459509903    sms   
365610    384166            not found  thailand        ais  459505105    wap   
365611  27966473            not found  thailand  aisgemezz         ge    sms   
365612    187009            not found  thailand        ais  459509903    sms   
365613  27906860            not found  thailand  aisgemezz         ge    sms   

             msisdn     status      cyc

# Cross-Sell

In [5]:
# Sort data by ID and Service Price
df.sort_values(by=['id', 'subs_date'], inplace=True)

# Create the label by grouping transactions by user ID and identify cross-sell events based on different service
df['cross_sell'] = (df['service_price'].lt(df.groupby('id')['service_price'].shift())) | \
                    df['service'].ne(df.groupby('id')['service'].shift()) & \
                    (df['profile_status'] == 'active')
                    
# Fill NaN values in the first row after sorting
df['cross_sell'].fillna(False, inplace=True)

In [6]:
df['cross_sell'].value_counts()

cross_sell
True     308288
False     57326
Name: count, dtype: int64

In [7]:
# Filter the DataFrame to include only rows where 'up_sell' is True
df_crosssell_true = df[df['cross_sell'] == True]

In [77]:
df_crosssell_true.describe()

,id,revenue,service_price,attempt_charging,success_billing,converted_service_price,converted_revenue
count,3.082880e+05,308288.000000,308288.000000,308288.000000,308288.000000,3.082880e+05,3.082880e+05
mean,6.675827e+05,4514.697134,81.448671,19.678216,2.785162,2.035453e+04,1.801329e+05
std,3.237222e+06,15343.271568,635.810649,26.746530,11.325157,1.487719e+05,4.895014e+06
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,8.002475e+04,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
50%,2.144625e+05,0.000000,6.000000,15.000000,0.000000,2.647380e+03,0.000000e+00
75%,3.583570e+05,6.000000,6.000000,26.000000,0.000000,2.647380e+03,1.500000e+03
max,6.008518e+07,506000.000000,10000.000000,4015.000000,1363.000000,1.242951e+06,5.742434e+08


In [8]:
revenue = df.revenue

In [9]:
revenue_mean = revenue.mean()
print(revenue_mean)

4351.89579720689


In [10]:
stats.jarque_bera(df['revenue'])

SignificanceResult(statistic=288786484.8566333, pvalue=0.0)

One-Sample T-Test

In [11]:
t_statistic, pvalue = stats.ttest_1samp(df_crosssell_true['revenue'], revenue_mean)

print('P-Value: ' + str(pvalue))
if pvalue < 0.05:
    print('Sufficient evidence to reject H0')
else:
    print('Insufficient evidence to reject H0')


P-Value: 3.83339424520665e-09
Sufficient evidence to reject H0


Because the pvalue is smaller than 0.05, H1 is accepted, namely the cross-sell strategies have significant effects on the revenue data.

# Upsell

In [12]:
# Sort data by ID and Service Price
df.sort_values(by=['id', 'service_price'], inplace=True)

# Create the label by grouping transactions by user ID and identify cross-sell events based on different service
df['up_sell'] = df['service_price'].gt(df.groupby('id')['service_price'].shift()) & \
                (df['profile_status'] == 'active')

# Fill NaN values in the first row after sorting
df['up_sell'].fillna(False, inplace=True)

In [13]:
df['up_sell'].value_counts()

up_sell
False    323365
True      42249
Name: count, dtype: int64

In [14]:
# Filter the DataFrame to include only rows where 'up_sell' is True
df_upsell_true = df[df['up_sell'] == True]

In [76]:
df_upsell_true.describe()

,id,revenue,service_price,attempt_charging,success_billing,converted_service_price,converted_revenue
count,4.224900e+04,42249.000000,42249.000000,42249.000000,42249.000000,4.224900e+04,4.224900e+04
mean,2.180689e+05,3900.395086,381.510379,27.864470,3.159743,7.058632e+04,5.802264e+05
std,1.658444e+05,16307.195401,1551.728036,60.322322,13.776219,2.800903e+05,8.069967e+06
min,4.000000e+00,0.000000,6.000000,0.000000,0.000000,7.500000e+02,0.000000e+00
25%,6.369200e+04,0.000000,6.000000,0.000000,0.000000,2.647380e+03,0.000000e+00
50%,1.768880e+05,0.000000,6.000000,15.000000,0.000000,2.647380e+03,0.000000e+00
75%,3.823370e+05,0.000000,6.000000,15.000000,0.000000,2.647380e+03,0.000000e+00
max,4.781612e+06,358000.000000,10000.000000,561.000000,556.000000,1.242951e+06,5.742434e+08


In [15]:
revenue = df.revenue

In [16]:
revenue_mean = revenue.mean()
print(revenue_mean)

4351.89579720689


In [17]:
stats.jarque_bera(df['revenue'])

SignificanceResult(statistic=288786484.8566329, pvalue=0.0)

One-Sample T-Test

In [18]:
# Perform a one-sample t-test on the 'revenue' column of df_upsell_true
t_statistic, pvalue = stats.ttest_1samp(df_upsell_true['revenue'], revenue_mean)

# Print the calculated p-value
print('P-Value: ' + str(pvalue))
if pvalue < 0.05:
    print('Sufficient evidence to reject H0')
else:
    print('Insufficient evidence to reject H0')


P-Value: 1.2714333576870448e-08
Sufficient evidence to reject H0


Because the pvalue is smaller than 0.05, H1 is accepted, namely the upsell strategies have significant effects on the revenue data.

# Independent T-Test

In [20]:
# Calculate t-statistics for mean revenue comparison
t_statistic, p_value = stats.ttest_ind(df_upsell_true['revenue'], df_crosssell_true['revenue'])

# Display the t-statistic and p-value
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

# Compare p-value against significance level (e.g., 0.05)
alpha = 0.05
if p_value < alpha:
    print("There is a significant difference between the mean revenues of the two datasets.")
else:
    print("There is no significant difference between the mean revenues of the two datasets.")

T-Statistic: -7.658048491763023
P-Value: 1.892588990695219e-14
There is a significant difference between the mean revenues of the two datasets.
